In [ ]:
import numpy as np
import xarray as xr

from eval_utilities import spatial_temporal_metrics as stm
from eval_utilities import visualization as vis
import matplotlib.pyplot as plt

In [ ]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
ds_ref = xr.open_zarr(CONFIG["path_ec_euro"]).sel(time=slice("2021", "2022"))
ds_mod = xr.open_zarr(CONFIG["path_xgb_v1"])

In [ ]:
mc = stm.Metrics(ds_mod, ds_ref, path="/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/spatial")

In [ ]:
shift = mc.phase_shift()

In [ ]:

for var in shift.variable.values:
    vis.vis_zarr_map(shift, var,
                     '/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/visualization/shift',1,99)

## Power Spectra

In [ ]:
common_vars = np.intersect1d(ds_ref.variable, ds_mod.variable)

In [ ]:
path_png = "/home/ch23/data_ch23/evalution_results/xgbosst_train_2010_2019_val_2020_2020_est_50_hist/visualization/spectrum"
for var in common_vars:
    vis.power_spectrum(ds_mod, ds_ref, var, path_png)